# Data Reduction to Top 100 Rated Movies

In [2]:
import pandas as pd
df_movies = pd.read_csv("data_full/movies.csv")
df_ratings = pd.read_csv("data_full/ratings.csv")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   title    62423 non-null  object
 2   genres   62423 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [ ]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


Get only top 100 rated movie

In [ ]:
movie_rating_count = pd.DataFrame(df_ratings['movieId'].value_counts())
df_movies_100 = pd.merge(df_movies, movie_rating_count.head(100).reset_index()['movieId'], on='movieId')
df_movies_100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  100 non-null    int64 
 1   title    100 non-null    object
 2   genres   100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [ ]:
df_ratings_100 = pd.merge(df_ratings, movie_rating_count.head(100).reset_index()['movieId'], on='movieId')
df_ratings_100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4171006 entries, 0 to 4171005
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 127.3 MB


In [ ]:
df_movies_100.to_csv("data/movies_100.csv", index=False)
df_ratings_100.to_csv("data/ratings_100.csv", index=False)

# Data Import

In [48]:
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

df_movies_100 = pd.read_csv("data/movies_100.csv")
df_ratings_100 = pd.read_csv("data/ratings_100.csv")

In [49]:
df_movies_100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  100 non-null    int64 
 1   title    100 non-null    object
 2   genres   100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [50]:
df_ratings_100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4171006 entries, 0 to 4171005
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 127.3 MB


In [51]:
import tensorflow_datasets as tfds

movies = tfds.load("movielens/100k-movies", split="train")

In [52]:
movies.take(2)

<_TakeDataset element_spec={'movie_genres': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [53]:
movies

<_PrefetchDataset element_spec={'movie_genres': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'movie_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [54]:
# import tensorflow as tf

# int_cols = df_movies_100.select_dtypes(include=int).columns
# # df_movies_100[int_cols] = df_movies_100[int_cols].astype('float32')

# rating_tensor = tf.data.Dataset.from_tensor_slices(df_movies_100)

In [56]:
df_movies_100[['title', 'year']] = df_movies_100['title'].str.extract(r'(.+)\((\d{4})\)')

In [90]:
df_movies_100['genres'] = df_movies_100['genres'].str.replace('|', ' ',)
df_movies_100

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995
1,32,Twelve Monkeys (a.k.a. 12 Monkeys),Mystery Sci-Fi Thriller,1995
2,34,Babe,Children Drama,1995
3,47,Seven (a.k.a. Se7en),Mystery Thriller,1995
4,50,"Usual Suspects, The",Crime Mystery Thriller,1995
...,...,...,...,...
95,7361,Eternal Sunshine of the Spotless Mind,Drama Romance Sci-Fi,2004
96,8961,"Incredibles, The",Action Adventure Animation Children Comedy,2004
97,33794,Batman Begins,Action Crime IMAX,2005
98,58559,"Dark Knight, The",Action Crime Drama IMAX,2008


In [92]:
import tensorflow as tf
ohe = tf.keras.layers.Hashing(
    num_bins=10,
    output_mode='one_hot')
ohe(df_movies_100['genres'].values[2])


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)>

In [100]:
multi = tf.keras.layers.TextVectorization(max_tokens=2000, output_mode='multi_hot')
multi.adapt(df_movies_100['title'].values)
multi(df_movies_100['title'].values[10])

<tf.Tensor: shape=(209,), dtype=float32, numpy=
array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)>

In [99]:
tf.keras.models.M

'Dumb & Dumber (Dumb and Dumber) '

'Toy Story '